In [ ]:
from datasets import load_dataset

In [ ]:
ds = load_dataset('Anthropic/hh-rlhf')

In [ ]:
len(ds['test'])

In [ ]:

def convert_format(sent):
    sents = sent.split('\n\n')
    humans = []
    assistants = []
    for i, s in enumerate(sents):
        if s.startswith('Human: '):
            humans.append(i)
        elif s.startswith('Assistant: '):
            assistants.append(i)
    if len(humans)!=len(assistants):
        # print(f'ERROR: {ds_index}')
        # print(item)
        return
    dialogue = []
    for i in range(len(humans)):
        human_sent = '\n'.join(sents[humans[i]:assistants[i]])
        if i+1<len(humans):
            assistant_sent = '\n'.join(sents[assistants[i]:humans[i+1]])
        else:
            assistant_sent = '\n'.join(sents[assistants[i]:])
        human_sent = human_sent[7:]
        assistant_sent = assistant_sent[11:]
        dialogue.append({'Human: ':human_sent,'Assistant: ':assistant_sent})
    return dialogue
def get_ds(ds):
    dialogues = []
    for ds_index,item in enumerate(ds):
        # sent = item['chosen']
        # print(sent)
        converted_chosen = convert_format(item['chosen'])
        converted_rejected = convert_format(item['rejected'])
        if converted_chosen is None or converted_rejected is None:
            continue
        dialogues.append({
            'chosen':converted_chosen,
            'rejected':converted_rejected,
        })
    return dialogues
def get_single_turn(ds):
    return [dialogue for dialogue in ds if len(dialogue['chosen'])==1 and len(dialogue['rejected'])==1]
def map_fn(sample):
    return {
        'prompt':sample['chosen'][0]['Human: '],
        'chosen':sample['chosen'][0]['Assistant: '],
        'rejected':sample['rejected'][0]['Assistant: '],
    }
new_ds = {
    'train':list(map(map_fn, get_single_turn(get_ds(ds['train'])))),
    'test':list(map(map_fn, get_single_turn(get_ds(ds['test'])))),
}

# dialogues

In [ ]:
new_ds

In [ ]:
import json
with open('/nfs2/wzt/deep-speed-chat/datasets/single_turn_rlhf/train.json','w') as f:
    json.dump(new_ds['train'],f)
with open('/nfs2/wzt/deep-speed-chat/datasets/single_turn_rlhf/test.json','w') as f:
    json.dump(new_ds['test'],f)

In [ ]:
my_ds = load_dataset("json",data_files={'train':'/nfs2/wzt/deep-speed-chat/datasets/single_turn_rlhf/train.json','test':'/nfs2/wzt/deep-speed-chat/datasets/single_turn_rlhf/test.json'})
my_ds

In [ ]:
my_ds['train'][0]

tmp = my_ds['train']
tmp

In [ ]:
tmp = [map_fn(item) for item in tmp]
tmp